# TODO

- [x] Code ran with flattened observation. How to run with nd-array observation? 
- [x] How to provide pytorch net that I trained earlier for fronzen lake.
- [ ] Use ray tuner for hyperparamter tuning.

In [1]:
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer

import pygame
import numpy as np

import torch
from IPython.display import clear_output
from IPython import display
import random
import matplotlib.pylab as plt
import copy
import time

from typing import List, Optional
from gym.envs.toy_text.frozen_lake import generate_random_map

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# redifne this map to put starting and ending point in random location
# TODO: Can we do this with decorators?
def newGenerate_random_map(size: int = 8, p: float = 0.8) -> List[str]:
    """Generates a random valid map (one that has a path from start to goal)
    Args:
        size: size of each side of the grid
        p: probability that a tile is frozen
    Returns:
        A random valid map
    """
    valid = False
    # DFS to check that it's a valid path.
    def is_valid(res,sx,sy):
        frontier, discovered = [], set()
        frontier.append((sx, sy))
        while frontier:
            r, c = frontier.pop()
            if not (r, c) in discovered:
                discovered.add((r, c))
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == "G":
                        return True
                    if res[r_new][c_new] != "H":
                        frontier.append((r_new, c_new))
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(["F", "H"], (size, size), p=[p, 1 - p])
        sx = np.random.randint(size); sy = np.random.randint(size)
        res[sx][sy] = "S"
        gx = np.random.randint(size); gy = np.random.randint(size)
        while res[gx][gy] == "S": # we don't want to overwrite the S
            gx = np.random.randint(size); gy = np.random.randint(size)
        res[gx][gy] = "G"   
        valid = is_valid(res,sx,sy)
    return ["".join(x) for x in res]

In [6]:
class FrozenLakeWrapped(gym.Env):
    def __init__(self,config):
        self.size = 6 # grid size
        nH = 10.    # desired number of hole: this is probabilistic
        pH = 1 - nH/(self.size**2) # probability of  grid being a hole
        desc = newGenerate_random_map(size= self.size, p= pH)
        self.env = gym.make('FrozenLake-v1', desc=desc, is_slippery=False)
        self.observation_space = gym.spaces.Box(0,1,shape=(3,self.size,self.size),dtype="int32")
        self.action_space = self.env.action_space
    def oneHot(self,s):
        x = np.zeros(self.size*self.size).astype("int32")
        x[s] = 1
        state_ = np.array([ x.reshape(self.size,self.size).astype("int32"),
                         np.array(self.env.desc == b"F").astype("int32"),
                         np.array(self.env.desc == b"G").astype("int32")
                          ])
        return state_.reshape(3,self.size,self.size,)# + np.random.rand(state_.size).reshape(state_.shape)/10.
    
    def reset(self):
        o = self.env.reset()
        return self.oneHot(o)

    def step(self, action):
        o, r, done, info = self.env.step(action)
        if done:
            if r > 0:
                newReward = 10
            else:
                newReward = -2
        else:
            newReward = -1
            
        return self.oneHot(o), newReward , done, info
    
    def render(self):
        self.env.render()

In [7]:
env = FrozenLakeWrapped({})
env.reset()
done = False

while not done:
    # plt.cla()
    display.clear_output(wait=True)

    obs, reward, done, _ = env.step(env.action_space.sample())
    env.render()
    time.sleep(0.1)
    # display.clear_output(wait=True)
    # display.display(plt.gcf())
    # plt.gcf()

  (Left)
FHFFHF
FFFFFF
FFFFFF
FFFFFG
HSFFFF
FHFFHH


In [8]:
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.framework import try_import_tf, try_import_torch

torch, nn = try_import_torch()

class MyTorchModel(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        """Build a simple [16, 16]-MLP (+ value branch)."""
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)
        self.device = torch.device("cpu")#"cuda"
        #                            if torch.cuda.is_available() else "cpu")

        self.mainLayer = nn.Sequential(
                nn.Conv2d(3,12,kernel_size=3,stride= 1,padding=1),
                nn.ReLU(),
                nn.Conv2d(12,24,kernel_size=3,stride=1,padding=1),
                nn.ReLU(),
                nn.Conv2d(24,36,kernel_size=3,stride=1,padding=1),
                nn.ReLU(),            
                nn.Flatten(start_dim=-3,end_dim=-1),
                nn.Linear(1296,1200),
                nn.ReLU(),
                nn.Linear(1200,64),
                nn.ReLU(),
        ).to(self.device)
        
        # Action logits output.
        self.layer_out = nn.Linear(64, num_outputs).to(self.device)

        # "Value"-branch (single node output).
        # Used by several RLlib algorithms (e.g. PPO) to calculate an observation's value.
        self.value_branch = nn.Linear(64, 1).to(self.device)
        self.cur_value = None

    def forward(self, input_dict, state, seq_lens):
        """Custom-define your forard pass logic here."""
        # Pass inputs through our 2 layers.
        layer_1_out = self.mainLayer(input_dict["obs"])
        logits = self.layer_out(layer_1_out)

        # Calculate the "value" of the observation and store it for
        # when `value_function` is called.
        self.cur_value = self.value_branch(layer_1_out).squeeze(-1)

        return logits, state

    def value_function(self):
        """Implement the value branch forward pass logic here:
        
        We will just return the already calculated `self.cur_value`.
        """
        assert self.cur_value is not None, "Must call `forward()` first!"
        return self.cur_value

In [9]:
size = 6
test_model_torch = MyTorchModel(
   obs_space=gym.spaces.Box(0,1,shape=(3,size,size,), dtype=np.int32),
   action_space=gym.spaces.Discrete(4),
   num_outputs=4,
   model_config={},
   name="MyModel",
)
#print("Torch-output={}".format(test_model_torch({"obs": torch.from_numpy(np.array([[0.5, 0.5]], dtype=np.float32))})))

obs = gym.spaces.Box(0,1,shape=(3,size,size)).sample()
test_model_torch({"obs": torch.from_numpy(obs)})

(tensor([-0.0702,  0.0210,  0.0069, -0.0752], grad_fn=<AddBackward0>), [])

In [10]:
trainer = DQNTrainer(
    config={
        "env": FrozenLakeWrapped,
        "framework": "torch",
        "env_config": {},
        "num_workers": 12,
        
            # "model": {
            # "fcnet_hiddens": [256, 800, 256],
            # "fcnet_activation": "relu",
        "model": {
        "custom_model": MyTorchModel,  # for torch users: "custom_model": MyTorchModel
        "custom_model_config": {
            #"layers": [128, 128],
        },
        },
    'num_envs_per_worker': 10,
    'gamma': 0.90,
    'lr': 0.001,
    'train_batch_size': 200,
    }
)

(RolloutWorker pid=3067793) 2022-07-15 12:29:27,040	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=3067793) 2022-07-15 12:29:27,085	ERROR worker.py:451 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=3067793, ip=10.13.62.8, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f7b28b31930>)
(RolloutWorker pid=3067793)   File "/home/ajit.kumar@SNU.IN/anaconda3/envs/udemy-rl/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 630, in __init__
(RolloutWorker pid=3067793)     self._build_policy_map(
(RolloutWorker pid=3067793)   File "/home/ajit.kumar@SNU.IN/anaconda3/envs/udemy-rl/lib/python3.10/site-packages/ray/rllib/evaluation

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=3067786, ip=10.13.62.8, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f0852771930>)
  File "/home/ajit.kumar@SNU.IN/anaconda3/envs/udemy-rl/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 630, in __init__
    self._build_policy_map(
  File "/home/ajit.kumar@SNU.IN/anaconda3/envs/udemy-rl/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1788, in _build_policy_map
    self.policy_map.create_policy(
  File "/home/ajit.kumar@SNU.IN/anaconda3/envs/udemy-rl/lib/python3.10/site-packages/ray/rllib/policy/policy_map.py", line 152, in create_policy
    self[policy_id] = class_(observation_space, action_space, merged_config)
  File "/home/ajit.kumar@SNU.IN/anaconda3/envs/udemy-rl/lib/python3.10/site-packages/ray/rllib/policy/policy_template.py", line 326, in __init__
    self._initialize_loss_from_du

In [ ]:
trainer = DQNTrainer(
    config={
        "env": "FrozenLake-v1",#FrozenLakeWrapped,
        "framework": "torch",
        "env_config": {},
        "num_workers": 12,
        
            "model": {
            "fcnet_hiddens": [256, 800, 256],
            "fcnet_activation": "relu",
        # "model": {
        # "custom_model": MyTorchModel,  # for torch users: "custom_model": MyTorchModel
        # "custom_model_config": {
        #     #"layers": [128, 128],
        # },
        },
    'num_envs_per_worker': 10,
    'gamma': 0.90,
    'lr': 0.001,
    'train_batch_size': 200,
    }
)

In [8]:
for i in range(50):    
    results = trainer.train()
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")

NameError: name 'trainer' is not defined

In [ ]:
# trainer.save("model")

In [ ]:
# trainer.load_checkpoint("model/checkpoint_000300/checkpoint-300")

In [28]:
won = 0
lost = 0
rewardList = []

for i in range(100):
    env = gym.make("FrozenLake-v1")
    #, #FrozenLakeWrapped({})
    # Get the initial observation (should be: [0.0] for the starting position).
    obs = env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    while not done:
        # plt.cla()
#        display.clear_output(wait=True)
        # print("num won: ", won, " played: ", i, "total reward: ", total_reward)
        # env.render()

        action = trainer.compute_single_action(obs)
        obs, reward, done, info = env.step(action)
        #print(obs, reward, done)
        total_reward += reward
        # time.sleep(0.1)
        
        # display.display(plt.gcf())
        # plt.gcf()
        if done: 
            rewardList.append(total_reward)
    if reward > 0 :
        won +=1 
      
    
#print(rewardList, np.mean(rewardList))
print( np.mean(rewardList))


0.23


In [ ]:
#TODO Why is testing average mean reward is very low compared to training mean reward

In [ ]:
for i in range(1):    
    results = trainer.train()
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")

In [23]:
import gym
from ray.rllib.agents.ppo import PPOTrainer


# Define your problem using python and openAI's gym API:
class SimpleCorridor(gym.Env):
    """Corridor in which an agent must learn to move right to reach the exit.

    ---------------------
    | S | 1 | 2 | 3 | G |   S=start; G=goal; corridor_length=5
    ---------------------

    Possible actions to chose from are: 0=left; 1=right
    Observations are floats indicating the current field index, e.g. 0.0 for
    starting position, 1.0 for the field next to the starting position, etc..
    Rewards are -0.1 for all steps, except when reaching the goal (+1.0).
    """

    def __init__(self, config):
        self.end_pos = config["corridor_length"]
        self.cur_pos = 0
        self.action_space = gym.spaces.Discrete(2)  # left and right
        self.observation_space = gym.spaces.Box(0.0, self.end_pos, shape=(1,))

    def reset(self):
        """Resets the episode and returns the initial observation of the new one."""
        self.cur_pos = 0
        # Return initial observation.
        return [self.cur_pos]

    def step(self, action):
        """Takes a single step in the episode given `action`

        Returns:
            New observation, reward, done-flag, info-dict (empty).
        """
        # Walk left.
        if action == 0 and self.cur_pos > 0:
            self.cur_pos -= 1
        # Walk right.
        elif action == 1:
            self.cur_pos += 1
        # Set `done` flag when end of corridor (goal) reached.
        done = self.cur_pos >= self.end_pos
        # +1 when goal reached, otherwise -1.
        reward = 1.0 if done else -0.1
        return [self.cur_pos], reward, done, {}


# Create an RLlib Trainer instance.
trainer = PPOTrainer(
    config={
        # Env class to use (here: our gym.Env sub-class from above).
        "env": SimpleCorridor,
        # Config dict to be passed to our custom env's constructor.
        "env_config": {
            # Use corridor with 20 fields (including S and G).
            "corridor_length": 20
        },
        # Parallelize environment rollouts.
        "num_workers": 3,
    }
)

# Train for n iterations and report results (mean episode rewards).
# Since we have to move at least 19 times in the env to reach the goal and
# each move gives us -0.1 reward (except the last move at the end: +1.0),
# we can expect to reach an optimal episode reward of -0.1*18 + 1.0 = -0.8
for i in range(5):
    results = trainer.train()
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")

# Perform inference (action computations) based on given env observations.
# Note that we are using a slightly different env here (len 10 instead of 20),
# however, this should still work as the agent has (hopefully) learned
# to "just always walk right!"
env = SimpleCorridor({"corridor_length": 10})
# Get the initial observation (should be: [0.0] for the starting position).
obs = env.reset()
done = False
total_reward = 0.0
# Play one episode.
while not done:
    # Compute a single action, given the current observation
    # from the environment.
    action = trainer.compute_single_action(obs)
    # Apply the computed action in the environment.
    obs, reward, done, info = env.step(action)
    # Sum up rewards for reporting purposes.
    total_reward += reward
# Report results.
print(f"Played 1 episode; total-reward={total_reward}")

2022-07-11 15:25:23,986	WARNING ppo.py:386 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=3 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1333.
(RolloutWorker pid=1068108) 2022-07-11 15:25:26,630	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=1068108) /home/ajit.kumar@SNU.IN/anaconda3/envs/udemy-rl/lib/python3.10/site-packages/gym/spaces/box.py:142: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=1068108)   logger.warn("Casting input x to numpy array.")
(RolloutWorker pid=1068107) 2022-07-11 15:25:26,586	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dicti

Iter: 0; avg. reward=-29.64285714285715
Iter: 1; avg. reward=-6.936752136752133
Iter: 2; avg. reward=-3.188829787234041
Iter: 3; avg. reward=-2.164227642276424
Iter: 4; avg. reward=-1.6125423728813568
Played 1 episode; total-reward=-0.7000000000000004


In [38]:
won = 0
lost = 0
rewardList = []

for i in range(100):

    env = SimpleCorridor({"corridor_length": 20})
    # Get the initial observation (should be: [0.0] for the starting position).
    obs = env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    while not done:
        # Compute a single action, given the current observation
        # from the environment.
        action = trainer.compute_single_action(obs)
        # Apply the computed action in the environment.
        obs, reward, done, info = env.step(action)
        # Sum up rewards for reporting purposes.
        total_reward += reward
        if done:
            rewardList.append(total_reward)
        
    # Report results.
#    print(f"Played 1 episode; total-reward={total_reward}")
print(np.mean(rewardList))

-1.3710000000000009
